# Brain Tumor Detection Model


## Importing Libraries


In [79]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import load_img
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

## Preparing Training Data


In [2]:
folder_path = "./dataset/train/"
filenames = []
categories = []

for category in os.listdir(folder_path):
    category_path = os.path.join(folder_path, category)
    
    if os.path.isdir(category_path):
        for filename in os.listdir(category_path):
            filenames.append(os.path.join(category, filename))
            categories.append(category)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['category'].value_counts().plot.bar()

## Visualizing Data


In [ ]:
sample = random.choice(filenames)
image = load_img("./dataset/train/"+sample)
plt.imshow(image)

# CNN(Convolutional Neural Network)


## Building CNN Model


In [ ]:
model = Sequential()

# Adding convolutional layers
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(200, 200, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

# Adding a second convolutional layer
model.add(Conv2D(36, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

### Call Backs


#### Early Stop

To prevent over fitting we will stop the learning after 10 epochs and val_loss value not decreased


In [88]:
earlystop = EarlyStopping(patience=10)

#### Learning Rate Reduction

We will reduce the learning rate when then accuracy not increase for 2 steps


In [89]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [90]:
callbacks = [earlystop, learning_rate_reduction]

### Preparing Data


In [112]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
train_df['category'].value_counts().plot.bar()

In [ ]:
validate_df['category'].value_counts().plot.bar()

### Train and Validation generator


In [115]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "./dataset/train/", 
    x_col='filename',
    y_col='category',
    target_size=(200,200),
    class_mode='binary',
    batch_size=15
)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./dataset/train/", 
    x_col='filename',
    y_col='category',
    target_size=(200,200),
    class_mode='binary',
    batch_size=15
)

### Visulaizing the output of data gen


In [ ]:
example_df = train_df.sample(n=2).reset_index(drop=True)
example_generator = train_datagen.flow_from_dataframe(
    example_df, 
    "./dataset/train/", 
    x_col='filename',
    y_col='category',
    target_size=(200,200),
    class_mode='binary'
)


In [ ]:
plt.figure(figsize=(12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()
plt.show()

### Fitting the Model


In [122]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator, 
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validate_df.shape[0]//15,
    steps_per_epoch=train_df.shape[0]//15,
    callbacks=callbacks
)

### Visualizing the Training history


In [ ]:
# Plotting the training history
plt.figure(figsize=(12, 8)) 

# Plot training & validation accuracy values
plt.subplot(2, 1, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

### Saving the CNN Model


In [ ]:
# Saving the Model
model.save('brain_tumor_detection_model.h5')

### Testing Data


In [126]:
test_folder_path = "./dataset/test"
test_filenames = []
test_categories = []

for category in os.listdir(test_folder_path):
    category_path = os.path.join(test_folder_path, category)
    
    if os.path.isdir(category_path):
        for filename in os.listdir(category_path):
            test_filenames.append(os.path.join(category, filename))
            test_categories.append(category)

test_df = pd.DataFrame({
    'filename': test_filenames,
    'category': test_categories
})

test_df['category'] = test_df['category'].map({'YES': 1, 'NO': 0})

### Predictions


In [ ]:
# Ensure the 'category' column has string values for binary classification
test_df['category'] = test_df['category'].replace({1: 'YES', 0: 'NO'})

steps = int(np.ceil(test_df.shape[0] / 15))

test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "./dataset/test", 
    x_col='filename',
    y_col='category',  # category column must have string values ('YES' or 'NO')
    target_size=(200,200),
    class_mode='binary',
    batch_size=15,
    shuffle=False
)

predictions = model.predict(test_generator, steps=steps)

# Convert the predicted values to 'YES' or 'NO'
test_df['predicted category'] = np.where(predictions > 0.5, 'YES', 'NO')


### Visualizing Predictions


In [ ]:
test_df.head()

In [ ]:
test_df.tail()

In [ ]:

plt.figure(figsize=(12, 4))

# Plot the first bar plot (predicted category)
plt.subplot(1, 2, 1)
test_df['predicted category'].value_counts().plot.bar()
plt.title('Predicted Categories')
plt.xlabel('Category')
plt.ylabel('Count')

# Plot the second bar plot (actual category)
plt.subplot(1, 2, 2)
test_df['category'].value_counts().plot.bar()
plt.title('Actual Categories')
plt.xlabel('Category')
plt.ylabel('Count')

plt.tight_layout()
plt.show()


### Confusion Matrix


In [ ]:
cm = confusion_matrix(test_df['category'], np.where(predictions > 0.5, 1, 0))

sns.heatmap(cm, annot=True, fmt='d', cmap='BuPu')

plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')

plt.show()

### Model Evaluation


In [ ]:
model.evaluate(test_generator)
